<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/es_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value

def check_lc_ta_dates(df,tw_index, start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date])
    df[end_date] = pd.to_datetime(df[end_date], errors='coerce')
    filtered = df.loc[pd.to_datetime(df[start_date]) > df[end_date], [tw_index, start_date,end_date]]
    print(filtered)
    if not filtered.empty:
        return filtered[[tw_index, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, old_file, new_file, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = fit_df(path_OLD,bill_cols)
        df_NEW = fit_df(path_NEW,bill_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_file}) vs ({new_file}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

    

Looking for differences betwwnn older and newest files in towerdb.

Adding new columns for UIP

In [ ]:
# Lendo o Ficheiro de input
path_uip=''
sheet = 'SiteLevel'
site_level = pd.read_excel(path_uip,sheet, header=[0,1,2])

head_1 = pd.MultiIndex.from_product([[''],['Numeric (in months)'],['Delay in Site Modification Projects', 'Delay in BTS Projects']])
df_1 = pd.DataFrame(columns=head_1)

head_2 = pd.MultiIndex.from_product([[''],['Numeric (in €)'],['Excess of Upgrade Capital Expenditure over Threshold']])
df_2 = pd.DataFrame(columns=head_2)

site_level = pd.concat([site_level, df_1, df_2], axis=1)
site_level.head(1)

Doing all checks in Towerdb File

In [63]:
def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):

    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    df = df.dropna(subset=[site_index], axis=0)

    return df

pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'Enhanced TowerDB'
skipr = 7
skipc = 3
towerdb = read_files(pathtw, sheet, skipr, skipc, 'Code')
towerdb = towerdb.rename(columns={'Categorization by Transmission Sys.1': 'Categorization by Transmission Sys_1'})
#towerdb.columns = lower_str(list(towerdb.columns))


pathmsa = '/content/TowerDB_Spain_20210731.csv'
msa = pd.read_csv(pathmsa, encoding='latin')
#msa.columns = lower_str(list(msa.columns))

col_order = list(msa.columns)

towerdb = towerdb[col_order]
#towerdb = towerdb.fillna('')

ints = ['Inhabitants', 'Altitude', '# of Lease Contracts', 'ORANGE', 'Telefonica', 'YOIGO']
for i in ints:
    towerdb[i] = towerdb[i].fillna(0)
    towerdb[i] = list(map(int, towerdb[i]))

dates_tw = ['First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date', 'Billing Trigger date']
for i in dates_tw:
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]
    
towerdb['Date of equipment removal (from MAR´21)'] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) and not isinstance(date_obj, str) else '' for date_obj in towerdb['Date of equipment removal (from MAR´21)']]

change_incorret(towerdb, 'Code', 'Sites_As_Metered_Estimated', 'Estimated model', 'Estimated Model' )

bill_cols = ['Code',\
             'Inhabitants',\
             'Categorization by Transmission Sys',\
             'Categorization by Site Type',\
             'Vodafone equipment giving Active Sharing to Orange',\
             'Bundled_Sites_Yes_No',\
			 'Wip_Site',\
			 'Region_For_Tax_Calculation',\
			 'Indoor_Outdoor_Categorization',\
			 'Bts_Site',\
			 'Sites_As_Metered_Estimated',\
			 'Indoor_Site_Any_Climate_Control',\
			 'Outdoor_Site_With__Power',\
			 'Bundled_Site_Categorization',\
			 'Strategic_Site',\
			 'Strategic_Site_Bucket',\
			 'Critical_Site',\
			 'CriticalSite_Beyond_10',\
			 'Active_Sharing_Arrangement',\
			 'Orange_Crossed_Site',\
			 'Das_Classification',\
			 'Macro_Core_Site_Yes_No',\
			 'Macro_Transmission_Hub_Yes_No',\
			 'Macro_Transmission_Hub_With_Shelters_Without_Shelters',\
			 'Transmission_With_Shelters_Without_Shelters',\
			 'Subsequent_Sharing_Arrangement',\
			 'First_Active_Sharing_Deployment_Type',\
             'First_Active_Sharing_Start_Date',\
             'First_Active_Sharing_End_Date',\
			 'Decommissioned_Site only for VF',\
			 'Sites_Fall_Under_2400',\
			 'Macro_CoreA_CoreB',\
			 'Billing Trigger date']
for i in bill_cols:
    towerdb[i] = towerdb[i].replace(['N/A', 'n/a',"0", '-', '_',' ', np.nan,'nan'], '')

towerdb = towerdb.fillna('')
towerdb.to_csv('/content/TowerDB_Spain_20210831.csv', encoding='windows-1252', index=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'Code'
tw_doer = 'Date of equipment removal (from MAR´21)'
tw_status = 'on air / active'
tw_bts = 'Bts_Site'
tw_bill = 'Billing Trigger date'
tw_wip = 'Wip_Site'
tw_decom = 'Decommissioned_Site only for VF'
#w_amount = 'Lease Contract - Current annual lease fee'
tw_critical = 'CriticalSite_Beyond_10'
msa_index = 'Code'
msa_bts = 'Bts_Site'
msa_doer = 'Date of equipment removal (from MAR´21)'
msa_status = 'on air / active'
msa_bill = 'Billing Trigger date'
msa_wip = 'Wip_Site'
msa_decom = 'Decommissioned_Site only for VF'
#msa_amount = 'Lease Contract - Current annual lease fee'
msa_critical = 'CriticalSite_Beyond_10'


Check columns received looking for missing columns that is gonna be used in rating engine

In [ ]:
"""Check Columns Received"""
def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

df_cols = check_columns_received(towerdb, col_order)
df_cols
#No columns missing

First Check - Dates Formats (dd/mm/YYYY) 

Columns: Date of equipment removal (from MAR´21)

In [11]:
"""You need to convert all values in cols for string format to check"""
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            #df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

# Columns to functions
dates_bill = [tw_index, tw_bill]
dates_doer = [tw_index, tw_doer]
#Columns to parser
bill=[tw_bill]
doer=[tw_doer]

actives_1 = towerdb[towerdb['on air / active']=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

#Checking columns for errors
actives_dates_errors = check_date_columns(actives_1, tw_index, tw_status, dates_bill, 2) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index,tw_status, dates_doer, 2) 
# No Actives sites with blank Date of Equipament Removal
print(actives_dates_errors)
print('\n')
print(no_actives_dates_errors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [ ]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = df_picklist[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

picklist_tw_general = {
    'Categorization by Transmission Sys' : ['Macro', 'Public DAS']
}
pick_col_general = ['Code', 'Categorization by Transmission Sys']

df_general_pick = check_picklist_v1(towerdb, tw_index, tw_status, pick_col_general, picklist_tw_general)
df_general_pick
# No errors


Fifth Check MoM Sites (BTS, decomissoned...)

In [13]:
"""Falta Coluna de Flag Indicating BTS Site no twerdb recebido"""
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):
   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         


actives = towerdb[towerdb[tw_status]=='In Service']
df_mom_bts = check_mom_bts(actives, tw_index, tw_bts, msa, msa_index, msa_bts)
# No one error df_mom_bts

decomiss = towerdb[towerdb[tw_status]=='Decommissioned']
df_mom_decom = check_mom_bts(decomiss, tw_index, tw_decom, msa, msa_index, msa_decom)
# No one error df_mom_decom

print(df_mom_bts)
print('\n')
print(df_mom_decom)

Empty DataFrame
Columns: [Code, Bts_Site]
Index: []


Empty DataFrame
Columns: [Code, Decommissioned_Site only for VF]
Index: []


Check Picklist and dates formats for In service sites




In [14]:
"""Check all picklist Columns"""
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = df_picklist[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

picklis_dict = {
    'Categorization by Transmission Sys': ['Macro', 'Public DAS'],
    'Categorization by Site Type': ['DAS passive','GBT','RTT'],
    'Sites_As_Metered_Estimated': ['Estimated Model','Metered Model'],
    #'Infrastructure ready (existing)/ to be ready (new)': [''],
    'Indoor_Site_Any_Climate_Control': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units'],
    'Bts_Site': ['Yes', 'No'],
    'Strategic_Site': ['Yes', 'No'],
    'Strategic_Site_Bucket': ['Yes - first 460', ''],
    'Critical_Site': ['Yes', 'No'],
    'CriticalSite_Beyond_10': ['Within 10%','Non Critical'],
    'Wip_Site': ['Yes', 'No'],
    'Decommissioned_Site only for VF': ['Yes', 'No'],
    'First_Active_Sharing_Deployment_Type': ['']
}

pick_cols = ['Code','Categorization by Transmission Sys','Categorization by Site Type','Sites_As_Metered_Estimated',\
             'Indoor_Site_Any_Climate_Control','Bts_Site',\
             'Strategic_Site','Strategic_Site_Bucket','Critical_Site','CriticalSite_Beyond_10','Wip_Site',\
             'Decommissioned_Site only for VF','First_Active_Sharing_Deployment_Type' ]

actives = towerdb[towerdb[tw_status]=='In Service']
df_in_service_picklist = check_picklist_v1(actives, tw_index, tw_status, pick_cols, picklis_dict)
df_in_service_picklist
# No errors

,on air / active
Code,


In [ ]:
#check dates in columns
def check_date_columns(df, df_index,status_col,columns, format=2):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

actives['First_Active_Sharing_End_Date'] = actives['First_Active_Sharing_End_Date'].fillna('')
in_service_cols = ['Code', 'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date']
df_in_service_dates = check_date_columns(actives, tw_index, tw_status, in_service_cols)
df_in_service_dates
# tem errors em df_in_service_dates sites ativos

Fifth Check BTS Flagged(Billing Trigger and Commercial)





In [62]:
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if t == 'doer':
        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
        return filtered[[tw_index, status_col, date_col]] 
    else:
        df_tw[date_col] = pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col].empty)|(df_tw[date_col]>current_date))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}', filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 

status = 'Yes'
df_bts_bill_errors= check_tw_bill_doer(towerdb, tw_index,tw_bill, tw_bts, status, 'bill')
df_bts_bill_errors
#No errors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Code,Bts_Site,Billing Trigger date
9105,95151,Yes,01/09/2021


In [20]:
""" BTS sites"""
path_uip = '/content/UserInput_Spain_202107 v31.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites(Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2).fillna('')
uip.columns = uip_names

msa_sites = [i for i in msa[msa_index]]
tw_sites = [i for i in towerdb[tw_index]]
uip_sites = [i for i in uip['Site_ID']]


In [ ]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = pd.to_datetime(df[bill_col], format='%d/%m/%Y', errors='coerce')
    df_site_bts = df[(df[bts_col]=='Yes')&(df[bill_col] > current_date)].fillna('')
    df_site_bts[bill_col] = list(map(lambda x: f'{x:%d/%m/%Y}', df_site_bts[bill_col]))
    df_site_bts = df_site_bts[[tw_index,status_col, bts_col, bill_col]]
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, df_site_bts

new_sites, df_bts_errors = check_new_sites(towerdb, tw_index, tw_bts, tw_bill,tw_status, msa_sites, tw_sites, uip_sites)

# New sites = 132
# Error on BTS sites out of UIP File
print(new_sites)
print('\n')
print(df_bts_errors)


Check Wip SItes

In [32]:
""" Wip Sites Check"""
def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, msa_wip):

    wip_msa = [i for i in df_msa[df_msa[msa_wip]=='Yes'][msa_index]]
    
    tw_wip_sites = [str(i) for i in df_tw[df_tw[wip_tw]=='Yes'][tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    #wip_out_tw_list = [i for i in tw_wip_sites if i not in wip_msa]

    return tw_wip_site_bts_flagged

df_wip_and_bts_flagged = check_wip(towerdb,tw_index, tw_wip, tw_bts, msa, msa_index, msa_wip)
df_wip_and_bts_flagged
#No errors

,Code,Wip_Site,Bts_Site


Check Decomissioned sites

In [31]:
def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
  
df_decom_sites = check_decommissioned(towerdb, tw_index, tw_decom, tw_doer)
df_decom_sites
#No errors

,Code,Decommissioned_Site only for VF,Date of equipment removal (from MAR´21)


Check Doer columns for in service sites

Should not to be in past or different of blank

In [ ]:
#COluna Doer tem valores fora do formato
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if t == 'doer':
        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
        return filtered[[tw_index, status_col, date_col]] 
    else:
        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, date_col]] 

actives[tw_doer] = actives[tw_doer].replace('Not removed', '')
df_doer = check_tw_bill_doer(actives, tw_index, tw_doer, tw_bts, 'Yes', 'doer')
df_doer
#No errors

*Tenth* - Check UIP Towerdb matches

In [75]:
def check_uip_tw(df_tw,tw_index, status_tw_col,decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    # Check for in service sites(towerdb) that not to be in UIS
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[decomiss_sites_in_uip[status_tw_col]=='Decommissioned']
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]                            

    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites(Annual)']!='']['Site_ID']]
    bts_tw_critical = [i for i in df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]]
    print(bts_tw_critical)
    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
    critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                right_on=tw_index)
    critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis,decomiss_sites_in_uip, bts_sites_out_uip, critical

uis_sites_not_in_towerdb, in_service_not_in_uis,decomiss_sites_in_uis, bts_sites_out_uis, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                              tw_decom, tw_bts,tw_critical,uip, uip_sites)
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(decomiss_sites_in_uis)
print('\n')
print(bts_sites_out_uis)
print('\n')
print(critical)


[]
    UIS In Month not active in TowerDB! on air / active
0                                  1184  Decommissioned
1                                 27266  Decommissioned
2                                 52208  Decommissioned
3                                 53060  Decommissioned
4                                 74808  Decommissioned
5                                173929  Decommissioned
6                                181769             WIP
7                                 95151             WIP
8                                 92098             WIP
9                                196617             NaN
10                                19445             NaN


Empty DataFrame
Columns: [TowerDB Sites out of UIS In Month!, on air / active]
Index: []


    Decomissioned Site in UIS File on air / active
1                             1184  Decommissioned
24                           27266  Decommissioned
36                           52208  Decommissioned


   UIS BTS not in TowerDB(

Commercial

In [73]:
def check_diffs_v2(path_current, path_last, sheet='Commercial'):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)

    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')

    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')
        
#cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type','Input_Value_actual', 'Input_Value_before' ,'Description/Instruction', 'Frequency of Update']
df_com = check_diffs_v2( '/content/UserInput_Spain_202107 v31.xlsx', '/content/UserInput_Spain_20210731.xlsx')
df_com

In [81]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')
        
logs = [['Dates Errors', actives_dates_errors],
        ['No Actives Dates Errors', no_actives_dates_errors],
        ['General Picklist Erros',df_general_pick],
        ['MSA BTS not in TowerDB',df_mom_bts],
        ['MSA Decom not in TowerDB', df_mom_decom],
        ['Active Picklist Errors', df_in_service_picklist],
        ['Actives Sites Dates Errors',df_in_service_dates],
        ['BTS Bill Dates Erros', df_bts_bill_errors],
        ['New Sites', new_sites],
        ['New Sites Bill Dates Erros',df_bts_errors], 
        ['Wip & BTS Flagged', df_wip_and_bts_flagged],
        ['Decom sites Dates Errors', df_decom_sites],
        ['DOER Errors', df_doer],
        ['UIS Sites not active in TowerDB', uis_sites_not_in_towerdb],
        ['TowerDB Sites out of UIS', in_service_not_in_uis],
        ['Decom Sites In UIS', decomiss_sites_in_uis],
        ['UIS BTS not in TowerDB(BTS)', bts_sites_out_uis],
        ['Sites Beyond 10% out UIS', critical],
        ['Comercial Differences', df_com]]

path_log = '/content/TowerDB_ES_Errors.xlsx'
general_log_erros(logs, path_log)

Lc Input

In [82]:
def date_parser(df, columns, format, type_dates='normal'):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'LC_INPUT_SPAIN'
skipr = 0
skipc = 0
lc = pd.read_excel(pathtw, sheet)
dates_lc = ['Inicio','Fin vigen/']
date_parser(lc,dates_lc,"%d/%m/%Y")
lc = lc.fillna('')
lc.head(3)
lc.to_csv('/content/LC_Input_Spain_20210831.csv', encoding='windows-1252', index=False)
#print([i for i in list(lc['Código']) if list(lc['Código']).count(i)>1])
#NO erros

In [ ]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            value = str(df.loc[site,column])
            if not value.__contains__(pattern):
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       

    return df_new

lc_cols = ['Código', '   Importe anual']
#lc['Importe anual'] = lc['   Importe anual'].astype(str)
#Python interpretou como float a coluna de Import Anual
df_lc_amount = check_amounts(lc, 'Código', lc_cols, '.')
df_lc_amount
# No errors

In [134]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    #Fit dates to correct format to compare
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y', errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date], format='%d/%m/%Y', errors='coerce')

    #filtering Dateframe for specific dates
    filtered = df.loc[(df[start_date] > df[end_date])|(df[start_date]=='')|(df[end_date]==''), [tw_index, start_date,end_date]]

    filtered[start_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[start_date]))
    filtered[end_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[end_date]))
    filtered = filtered[[tw_index, start_date,end_date]] 
    return filtered

df_lc_dates = check_lc_ta_dates(lc,'Código', 'Inicio','Fin vigen/')
df_lc_dates

,Código,Inicio,Fin vigen/


In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

logs_lc = [['Amount Errors', df_lc_amount],
        ['Dates Errors', df_lc_dates]]
        
path_log_lc = '/content/LC_ES_Errors.xlsx'
general_log_erros(logs_lc, path_log_lc)

TA OSP

In [96]:
pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'TA_Input_Lease_OSP_Spain'
skipr = 0
skipc = 0
ta_bill = ['Código', 'Importe anual','Inicio','F/cie/tec/']
ta_osp = pd.read_excel(pathtw, sheet)

dates_ta_osp = ['Inicio','F/cie/tec/']
for i in dates_ta_osp:
    ta_osp[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta_osp[i]]

ta_osp = ta_osp.fillna('')
ta_osp = ta_osp[~(ta_osp['Importe anual']=="")]

ta_osp.to_csv('/content/TA_Input_Lease_OSP_Spain_20210831.csv', encoding = 'windows-1252',index=False)
ta_osp.head(2)
#ta_osp = ta_osp[ta_osp['Inicio']!=""]

,Zona,Código,Towering,Nombre Loc,Nombre 1,Número de objeto,Por mes,Period.,Importe anual,F/cie/tec/,Siguiente,Fin+tardío,Válido de,Validez a,Status actual,NºAlquiler,Alquiler Anterior,Den.cl.contrato,Denomin.forma pago,Clase de condición,Objetivo cond.,CeBe,Comentario Central,Comentario Zona,Código Municipio,Provincia,Municipio,Crea el,Inicio,Entorno,Gpo.autoriz.,IDObjDist,ID objeto
0,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [115]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            value = str(df.loc[site,column])
            if not value.__contains__(pattern):
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'
                else:
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    return df_new


ta_osp_cols = ['Código', 'Importe anual']  

ta_osp['Importe anual'] = ta_osp['Importe anual'].astype(str)

df_ta_osp_amount = check_amounts(ta_osp, 'Código', ta_osp_cols, '.')
df_ta_osp_amount

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


""


In [116]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    #Fit dates to correct format to compare
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y', errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date], format='%d/%m/%Y', errors='coerce')

    #filtering Dateframe for specific dates
    filtered = df.loc[(df[start_date] > df[end_date])|(df[start_date]=='')|(df[end_date]==''), [tw_index, start_date,end_date]]

    filtered[start_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[start_date]))
    filtered[end_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[end_date]))
    filtered = filtered[[tw_index, start_date,end_date]] 
    return filtered
    
df_ta_osp_dates = check_lc_ta_dates(ta_osp,'Código', 'Inicio','F/cie/tec/')
df_ta_osp_dates


,Código,Inicio,F/cie/tec/


In [109]:
pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'TA_Input_Lease_TME_Spain'
skipr = 0
skipc = 0

ta_tme = pd.read_excel(pathtw, sheet)
dates_ta_tme = ['Inicio','F/cie/tec/']
for i in dates_ta_tme:
    ta_tme[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta_tme[i]]

ta_tme = ta_tme.fillna('')
ta_tme = ta_tme[~(ta_tme['Importe anual']=="")]

ta_tme.to_csv('/content/TA_Input_Lease_TME_Spain_20210831.csv', encoding = 'windows-1252',index=False)
#ta_tme = ta_tme[ta_tme['Inicio']!=""]

In [117]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    #Fit dates to correct format to compare
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y', errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date], format='%d/%m/%Y', errors='coerce')

    #filtering Dateframe for specific dates
    filtered = df.loc[(df[start_date] > df[end_date])|(df[start_date]=='')|(df[end_date]==''), [tw_index, start_date,end_date]]

    filtered[start_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[start_date]))
    filtered[end_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[end_date]))
    filtered = filtered[[tw_index, start_date,end_date]] 
    return filtered
      
df_ta_tme_dates = check_lc_ta_dates(ta_tme,'Código', 'Inicio','F/cie/tec/')
df_ta_tme_dates

,Código,Inicio,F/cie/tec/


In [119]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            value = str(df.loc[site,column])
            if not value.__contains__(pattern):
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'
                else:
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       

    return df_new


ta_tme_cols = ['Código', 'Importe anual'] 
am= ['Importe anual']
#ta_tme['Importe anual'] = ta_tme['Importe anual'].astype(str)
df_ta_tme_amount = check_amounts(ta_tme, 'Código', ta_tme_cols, '.')
df_ta_tme_amount
# No errors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


""


In [ ]:
pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'TA_Input_Lease_MM_Spain'
skipr = 0
skipc = 0
ta_mm = pd.read_excel(pathtw, sheet)

dates_ta_mm = ['Inicio','F/cie/tec/']
for i in dates_ta_mm:
    ta_mm[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta_mm[i]]

ta_mm = ta_mm.fillna('')
ta_mm = ta_mm[~(ta_mm['Importe anual']=="")]

ta_mm.to_csv('/content/TA_Input_Lease_MM_Spain_20210831.csv', encoding = 'windows-1252',index=False)
ta_mm.head(2)
#ta_mm = ta_mm[ta_mm['Inicio']!=""]

#NO erros

In [121]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    #Fit dates to correct format to compare
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y', errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date], format='%d/%m/%Y', errors='coerce')

    #filtering Dateframe for specific dates
    filtered = df.loc[(df[start_date] > df[end_date])|(df[start_date]=='')|(df[end_date]==''), [tw_index, start_date,end_date]]

    filtered[start_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[start_date]))
    filtered[end_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[end_date]))
    filtered = filtered[[tw_index, start_date,end_date]] 
    return filtered
    
df_ta_mm_dates = check_lc_ta_dates(ta_mm,'Código', 'Inicio','F/cie/tec/')
df_ta_mm_dates 


,Código,Inicio,F/cie/tec/


In [122]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            value = str(df.loc[site,column])
            if not value.__contains__(pattern):
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'
                else:
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    return df_new


ta_mm_cols = ['Código', 'Importe anual']  

ta_mm['Importe anual'] = ta_mm['Importe anual'].astype(str)
df_ta_mm_amount = check_amounts(ta_mm, 'Código', ta_mm_cols, '.')
df_ta_mm_amount
# No errors


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


""


In [ ]:
pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'TA_Input_Lease_Others_Spain'
skipr = 0
skipc = 0
ta_others = pd.read_excel(pathtw, sheet)

dates_ta_ot = ['Inicio','F/cie/tec/']
for i in dates_ta_ot:
    ta_others[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta_others[i]]

ta_others = ta_others.fillna('')
ta_others = ta_others[~(ta_others['Importe anual']=="")]

ta_others.to_csv('/content/TA_Input_Lease_Other_Spain_20210831.csv', encoding = 'windows-1252',index=False)
ta_others.head(3)
#ta_others = ta_others.fillna('')
#ta_others = ta_others[ta_others['Inicio']!=""]



In [124]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):
    #Fit dates to correct format to compare
    df[start_date] = pd.to_datetime(df[start_date], format='%d/%m/%Y', errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date], format='%d/%m/%Y', errors='coerce')

    #filtering Dateframe for specific dates
    filtered = df.loc[(df[start_date] > df[end_date])|(df[start_date]=='')|(df[end_date]==''), [tw_index, start_date,end_date]]

    filtered[start_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[start_date]))
    filtered[end_date] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[end_date]))
    filtered = filtered[[tw_index, start_date,end_date]] 
    return filtered
    
df_ta_o_dates = check_lc_ta_dates(ta_others,'Código', 'Inicio','F/cie/tec/')
df_ta_o_dates
#NO erros

,Código,Inicio,F/cie/tec/


In [126]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            value = str(df.loc[site,column])
            if not value.__contains__(pattern):
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'
                else:
                    if value == '':
                        df_new.loc[site,column] = 'Blank Value'
                    else:
                        df_new.loc[site,column] = f'Incorret Format: {value}'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    return df_new

ta_o_cols = ['Código', 'Importe anual']

df_ta_o_amount = check_amounts(ta_others, 'Código', ta_o_cols, '.')
df_ta_o_amount
# No errors


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


""


In [130]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

logs_ta =[['OSP Amount Errors', df_ta_osp_amount],
        ['OSP Dates Erros',df_ta_osp_dates],
        ['TME Amount Errors', df_ta_tme_dates],
        ['TME Dates Erros',df_ta_tme_amount],
        ['MM Amount Errors', df_ta_mm_dates],
        ['MM Dates Erros',df_ta_mm_amount],
        ['Others Amount Errors', df_ta_o_dates],
        ['Others Dates Erros',df_ta_o_amount]]
        
path_ta = '/content/TA_ES_Errors.xlsx'
general_log_erros(logs_ta, path_ta)

No one log to write!
